<a href="https://colab.research.google.com/github/pankb1/Big-Data/blob/main/Ans_2_wine_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=19976c34ee35e5f91f8473f0f5381b7af57b0716a1d80ff6cfc64346aee07e69
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
import pyspark


In [162]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler



In [11]:
spark = SparkSession.builder.master('local').appName('dl').getOrCreate()

In [32]:
df = spark.read.csv("wine-class.csv", header=True, inferSchema=True)

In [33]:
df

DataFrame[class: int, alcohol: double,  malic_acid: double,  ash: double,  alcalinity_of_ash: double,  magnesium: int,  total_phenols: double,  flavanoids: double,  nonflavanoid_phenols: double,  proanthocyanins: double,  color_intensity: double,  hue: double,  od280/od315_of_diluted_wines: double,  proline: int]

In [34]:
df.show()

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|    1|  14.23|       1.71|2.43|              15.6|       127|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|    1065|
|    1|   13.2|       1.78|2.14|              11.2|       100|          2.65|       2.76|                 0.26|            1.28|            4.38|1.05|                          3.4|    1050|
|    1|  13.16|       2.36|2.67|              18.6

In [43]:
df.printSchema()

root
 |-- class: integer (nullable = true)
 |-- alcohol: double (nullable = true)
 |--  malic_acid: double (nullable = true)
 |--  ash: double (nullable = true)
 |--  alcalinity_of_ash: double (nullable = true)
 |--  magnesium: integer (nullable = true)
 |--  total_phenols: double (nullable = true)
 |--  flavanoids: double (nullable = true)
 |--  nonflavanoid_phenols: double (nullable = true)
 |--  proanthocyanins: double (nullable = true)
 |--  color_intensity: double (nullable = true)
 |--  hue: double (nullable = true)
 |--  od280/od315_of_diluted_wines: double (nullable = true)
 |--  proline: integer (nullable = true)



In [46]:
for col in df.columns:
  df =df.withColumn(col,df[col].cast('double'))

In [157]:
# input column
input_cols = df.columns[1:-1]

In [158]:
df.select(input_cols).show()

+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+
| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines|
+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+
|       1.71|2.43|              15.6|     127.0|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|
|       1.78|2.14|              11.2|     100.0|          2.65|       2.76|                 0.26|            1.28|            4.38|1.05|                          3.4|
|       2.36|2.67|              18.6|     101.0|           2.8|       3.24|                  0.3|            2.81|            5.68|1.03|                         3.17

In [53]:
# output column
output_cols = df.columns[:1]

In [55]:
df.select(output_cols).show()

+-----+
|class|
+-----+
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
+-----+
only showing top 20 rows



In [56]:
df.countdf.count

<bound method DataFrame.count of DataFrame[class: double, alcohol: double,  malic_acid: double,  ash: double,  alcalinity_of_ash: double,  magnesium: double,  total_phenols: double,  flavanoids: double,  nonflavanoid_phenols: double,  proanthocyanins: double,  color_intensity: double,  hue: double,  od280/od315_of_diluted_wines: double,  proline: double]>

In [57]:
pdf = df.toPandas()
pdf

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,1.0,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,1.0,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,1.0,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,1.0,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3.0,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,3.0,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,3.0,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,3.0,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [59]:
df.groupby('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|  1.0|   59|
|  3.0|   48|
|  2.0|   71|
+-----+-----+



In [163]:
pdf.isnull().sum()

class                            0
alcohol                          0
 malic_acid                      0
 ash                             0
 alcalinity_of_ash               0
 magnesium                       0
 total_phenols                   0
 flavanoids                      0
 nonflavanoid_phenols            0
 proanthocyanins                 0
 color_intensity                 0
 hue                             0
 od280/od315_of_diluted_wines    0
 proline                         0
dtype: int64

In [61]:
from pyspark.sql.functions import isnull, when, count, col

In [62]:
#missing values
df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|    0|      0|          0|   0|                 0|         0|             0|          0|                    0|               0|               0|   0|                            0|       0|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+



In [63]:
from pyspark.ml.feature import Imputer, MinMaxScaler

In [74]:
imputed_col = ['f_{}'.format(i+1) for i in range(11)]

In [72]:
imputed_col

['f_1']

In [75]:
model = Imputer(strategy = 'mean', missingValue=None, inputCols=input_cols,outputCols=imputed_col).fit(df)

In [77]:
impute_data = model.transform(df)

In [78]:
impute_data.show()

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+----+----+----+-----+----+----+----+----+----+----+----+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline| f_1| f_2| f_3|  f_4| f_5| f_6| f_7| f_8| f_9|f_10|f_11|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+----+----+----+-----+----+----+----+----+----+----+----+
|  1.0|  14.23|       1.71|2.43|              15.6|     127.0|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|  1065.0|1.71|2.43|15.6|127.0| 2.8|3.06|0.28|2.29|5.64|1.04|3.92|
|  1.0|   13.2| 

In [79]:
assemble = VectorAssembler(inputCols=imputed_col, outputCol='assembled_features')

In [83]:
assemble_data = assemble.transform(impute_data)

In [85]:
assemble_data.show()

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+----+----+----+-----+----+----+----+----+----+----+----+--------------------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline| f_1| f_2| f_3|  f_4| f_5| f_6| f_7| f_8| f_9|f_10|f_11|  assembled_features|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+----+----+----+-----+----+----+----+----+----+----+----+--------------------+
|  1.0|  14.23|       1.71|2.43|              15.6|     127.0|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|  1065.0|1.71|2.43|

In [86]:
scaler = MinMaxScaler(min=0.0, max=1.0,inputCol='assembled_features',outputCol='features')

In [87]:
scaler_data = scaler.fit(assemble_data).transform(assemble_data)

In [88]:
scaler_data.select('features').show()

+--------------------+
|            features|
+--------------------+
|[0.19169960474308...|
|[0.20553359683794...|
|[0.32015810276679...|
|[0.23913043478260...|
|[0.36561264822134...|
|[0.20158102766798...|
|[0.22332015810276...|
|[0.27865612648221...|
|[0.17786561264822...|
|[0.12055335968379...|
|[0.28063241106719...|
|[0.14624505928853...|
|[0.19565217391304...|
|[0.19565217391304...|
|[0.22332015810276...|
|[0.21146245059288...|
|[0.23320158102766...|
|[0.16403162055335...|
|[0.16798418972332...|
|[0.46640316205533...|
+--------------------+
only showing top 20 rows



In [91]:
from pyspark.ml.classification import RandomForestClassifier

In [92]:
rf = RandomForestClassifier(labelCol='Outcome',featuresCol='features')

In [117]:
from typing_extensions import Required
# Assemble the features
Required_features = ['alcohol']

In [118]:
pdf[Required_features]

,alcohol
0,14.23
1,13.20
2,13.16
3,14.37
4,13.24
...,...
173,13.71
174,13.40
175,13.27
176,13.17


In [119]:
 #import the vector assembler class
from pyspark.ml.feature import VectorAssembler

In [122]:
# create the object
assembler = VectorAssembler(inputCols=Required_features,outputCol='features')

In [123]:
transformed_data = assembler.transform(df)

In [140]:
training_data, test_data = transformed_data.randomSplit([0.7,0.3])


In [183]:
rf = RandomForestClassifier(labelCol="class", featuresCol="features", numTrees=10)


In [177]:
training_data.count()

133

In [184]:
model = rf.fit(training_data)

In [186]:
predictions = model.transform(test_data)

In [187]:
predictions.show()

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+--------+--------------------+--------------------+----------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|features|       rawPrediction|         probability|prediction|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+--------+--------------------+--------------------+----------+
|  1.0|  13.05|       1.73|2.04|              12.4|      92.0|          2.72|       3.27|                 0.17|            2.91|             7.2|1.12|                         2.91|  1150.0| [13.05]|[0.0,5.2730519480...|[0.0,0.5273051948...|    

In [188]:
predictions.select('prediction').toPandas()

,prediction
0,1.0
1,2.0
2,3.0
3,3.0
4,3.0
5,3.0
6,1.0
7,1.0
8,3.0
9,3.0


In [192]:
pred = predictions.select('prediction').toPandas()

In [194]:
actual = test_data.select('class').toPandas()

In [195]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [198]:
eval = MulticlassClassificationEvaluator(labelCol='class',metricName='accuracy')

In [199]:
print('Accuracy: ', eval.evaluate(predictions))

Accuracy:  0.6
